In [2]:
import pandas as pd

In [3]:
df_items = pd.read_parquet('../data/interim/parquet/final_items.parquet')
df_games = pd.read_parquet('../data/interim/parquet/final_games.parquet')
df_reviews = pd.read_parquet('../data/interim/parquet/final_reviews.parquet')

In [4]:
df_items['item_id'] = df_items['item_id'].astype(int)

In [7]:
import pandas as pd

# Asumiendo que df_items, df_reviews y df_games son tus DataFrames.

# Primer merge: unir df_items y df_games en 'item_id' y 'id'
merged_df_1 = pd.merge(df_items, df_games, left_on='item_id', right_on='id')

# Segundo merge: unir el DataFrame resultante con df_reviews en 'user_id'
final_merged_df = pd.merge(merged_df_1, df_reviews, on='user_id')

# Ahora final_merged_df tiene toda la información combinada de los tres datasets

final_merged_df.to_parquet('../data/interim/parquet/merge.parquet', index=False)
final_merged_df.to_csv('../data/interim/csv/merge.csv', index=False)

In [8]:
final_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6562912 entries, 0 to 6562911
Data columns (total 51 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   user_id                object 
 1   steam_id               object 
 2   item_id_x              int32  
 3   item_name              object 
 4   playtime_forever       int64  
 5   app_name               object 
 6   id                     int32  
 7   price                  float64
 8   release_year           int32  
 9   Indie                  int32  
 10  Action                 int32  
 11  Adventure              int32  
 12  Casual                 int32  
 13  Simulation             int32  
 14  Strategy               int32  
 15  RPG                    int32  
 16  Singleplayer           int32  
 17  Free to Play           int32  
 18  Multiplayer            int32  
 19  Great Soundtrack       int32  
 20  Puzzle                 int32  
 21  Early Access           int32  
 22  2D                

In [9]:
# Definir el nuevo orden de las columnas
new_column_order = [
    'user_id', 'steam_id', 'item_id_x', 'item_name', 'playtime_forever',
    'app_name', 'id', 'price', 'release_year', 'funny', 'posted',
    'item_id_y', 'helpful', 'recommend', 'sentiment_analysis',
    # Mover las columnas de géneros al final
    'Indie', 'Action', 'Adventure', 'Casual', 'Simulation', 'Strategy', 'RPG',
    'Singleplayer', 'Free to Play', 'Multiplayer', 'Great Soundtrack', 'Puzzle',
    'Early Access', '2D', 'Atmospheric', 'VR', 'Sports', 'Platformer',
    'Story Rich', 'Sci-fi', 'Fantasy', 'Horror', 'Open World', 'Difficult',
    'Anime', 'Massively Multiplayer', 'Pixel Graphics', 'Co-op', 'Shooter',
    'Racing', 'Female Protagonist', 'Funny', 'First-Person', 'FPS', 'Sandbox',
    'Arcade'
]

# Reorganizar las columnas
final_merged_df = final_merged_df.reindex(columns=new_column_order)


In [10]:
# Seleccionar las columnas relevantes
genre_playtime_df = final_merged_df[['user_id', 'playtime_forever', 'release_year', 'Indie', 'Action', 'Adventure', 'Casual', 'Simulation', 'Strategy', 'RPG', 'Singleplayer', 'Free to Play', 'Multiplayer', 'Great Soundtrack', 'Puzzle', 'Early Access', '2D', 'Atmospheric', 'VR', 'Sports', 'Platformer', 'Story Rich', 'Sci-fi', 'Fantasy', 'Horror', 'Open World', 'Difficult', 'Anime', 'Massively Multiplayer', 'Pixel Graphics', 'Co-op', 'Shooter', 'Racing', 'Female Protagonist', 'Funny', 'First-Person', 'FPS', 'Sandbox', 'Arcade']]

# Guardar el DataFrame
genre_playtime_df.to_parquet('../data/interim/parquet/genre_playtime.parquet')
genre_playtime_df.to_csv('../data/interim/csv/genre_playtime.csv')


In [11]:
# Seleccionar las columnas relevantes
recommendations_df = final_merged_df[['app_name', 'release_year', 'recommend', 'sentiment_analysis']]

# Guardar el DataFrame
recommendations_df.to_parquet('../data/interim/parquet/recommendations.parquet')
recommendations_df.to_csv('../data/interim/csv/recommendations.csv')


In [12]:
def PlayTimeGenre(genero: str):
    # Cargar el DataFrame genre_playtime_df
    genre_playtime_df = pd.read_parquet('../data/interim/parquet/genre_playtime.parquet')
    
    # Filtrar solo las filas donde el género especificado es 1 (presente)
    genre_df = genre_playtime_df[genre_playtime_df[genero] == 1]
    
    # Agrupar por año de lanzamiento y sumar el tiempo de juego
    year_playtime = genre_df.groupby('release_year')['playtime_forever'].sum()
    
    # Encontrar el año con el máximo tiempo de juego
    max_playtime_year = year_playtime.idxmax()
    
    return {"Año de lanzamiento con más horas jugadas para Género " + genero : max_playtime_year}


In [14]:
PlayTimeGenre('Indie')

{'Año de lanzamiento con más horas jugadas para Género Indie': 2006}

In [15]:
def UserForGenre(genero: str):
    # Cargar el DataFrame genre_playtime_df
    genre_playtime_df = pd.read_parquet('../data/interim/parquet/genre_playtime.parquet')
    
    # Filtrar solo las filas donde el género especificado es 1 (presente)
    genre_df = genre_playtime_df[genre_playtime_df[genero] == 1]
    
    # Agrupar por usuario y sumar el tiempo de juego
    user_playtime = genre_df.groupby('user_id')['playtime_forever'].sum()
    
    # Encontrar el usuario con el máximo tiempo de juego
    max_playtime_user = user_playtime.idxmax()
    
    # Filtrar las filas para el usuario específico
    user_df = genre_df[genre_df['user_id'] == max_playtime_user]
    
    # Agrupar por año de lanzamiento y sumar el tiempo de juego
    user_year_playtime = user_df.groupby('release_year')['playtime_forever'].sum().reset_index()
    
    # Crear la lista de horas jugadas por año
    hours_played = user_year_playtime.to_dict('records')
    
    return {"Usuario con más horas jugadas para Género " + genero : max_playtime_user, "Horas jugadas": hours_played}



In [16]:
UserForGenre('Indie')

{'Usuario con más horas jugadas para Género Indie': '76561198063648921',
 'Horas jugadas': [{'release_year': -1, 'playtime_forever': 4368},
  {'release_year': 1999, 'playtime_forever': 12816},
  {'release_year': 2003, 'playtime_forever': 114576},
  {'release_year': 2006, 'playtime_forever': 6938448},
  {'release_year': 2010, 'playtime_forever': 23016},
  {'release_year': 2011, 'playtime_forever': 203520},
  {'release_year': 2012, 'playtime_forever': 181512},
  {'release_year': 2013, 'playtime_forever': 192768},
  {'release_year': 2014, 'playtime_forever': 354144},
  {'release_year': 2015, 'playtime_forever': 253632},
  {'release_year': 2016, 'playtime_forever': 348528},
  {'release_year': 2017, 'playtime_forever': 12528}]}

In [17]:
def UsersRecommend(año: int):
    # Cargar el DataFrame recommendations_df
    recommendations_df = pd.read_parquet('../data/interim/parquet/recommendations.parquet')
    
    # Filtrar las filas para el año especificado y las recomendaciones positivas
    year_recommend_df = recommendations_df[(recommendations_df['release_year'] == año) & (recommendations_df['recommend'] == True)]
    
    # Agrupar por nombre de la aplicación y contar las recomendaciones
    game_recommend_count = year_recommend_df.groupby('app_name').size()
    
    # Ordenar los juegos por la cantidad de recomendaciones y tomar los 3 primeros
    top_3_games = game_recommend_count.sort_values(ascending=False).head(3).index.tolist()
    
    return [{"Puesto 1": top_3_games[0]}, {"Puesto 2": top_3_games[1]}, {"Puesto 3": top_3_games[2]}]


In [22]:
UsersRecommend(2018)

[{'Puesto 1': 'Lost Moon'},
 {'Puesto 2': 'Divinia Chronicles: Relics of Gan-Ti'},
 {'Puesto 3': 'Empires Of Creation'}]

In [24]:
def UsersNotRecommend(año: int):
    # Reutilizar la función UsersRecommend, pero con el filtro de recomendación negativa
    recommendations_df = pd.read_parquet('../data/interim/parquet/recommendations.parquet')
    year_not_recommend_df = recommendations_df[(recommendations_df['release_year'] == año) & (recommendations_df['recommend'] == False)]
    game_not_recommend_count = year_not_recommend_df.groupby('app_name').size()
    bottom_3_games = game_not_recommend_count.sort_values().head(3).index.tolist()
    return [{"Puesto 1": bottom_3_games[0]}, {"Puesto 2": bottom_3_games[1]}, {"Puesto 3": bottom_3_games[2]}]


In [25]:
UsersNotRecommend(2001)

[{'Puesto 1': 'The Mystery of the Druids'},
 {'Puesto 2': "Kohan: Ahriman's Gift"},
 {'Puesto 3': 'Kohan: Immortal Sovereigns'}]

In [ ]:
def sentiment_analysis(año: int):
    # Cargar el DataFrame recommendations_df
    recommendations_df = pd.read_parquet('../data/interim/parquet/recommendations.parquet')
    
    # Filtrar las filas para el año especificado
    year_sentiment_df = recommendations_df[recommendations_df['release_year'] == año]
    
    # Contar los registros de análisis de sentimiento
    sentiment_counts = year_sentiment_df['sentiment_analysis'].value_counts()
    
    return {"Negative": sentiment_counts.get(0, 0),
            "Neutral": sentiment_counts.get(1, 0),
            "Positive": sentiment_counts.get(2, 0)}
